# Patch daily with jura

In [ ]:
import os
import numpy as np
from astropy.table import Table, join
from astropy.io import fits
from specprodDB.util import cameraid

In [ ]:
daily_tiles_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'daily', 'tiles-daily.csv')
daily_exposures_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'daily', 'exposures-daily.fits')
jura_tiles_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'jura', 'tiles-jura.csv')
jura_exposures_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'jura', 'exposures-jura.fits')
daily_tiles = Table.read(daily_tiles_file, format='ascii.csv')
daily_exposures = Table.read(daily_exposures_file, format='fits', hdu='EXPOSURES')
daily_frames = Table.read(daily_exposures_file, format='fits', hdu='FRAMES')
jura_tiles = Table.read(jura_tiles_file, format='ascii.csv')
jura_exposures = Table.read(jura_exposures_file, format='fits', hdu='EXPOSURES')
jura_frames = Table.read(jura_exposures_file, format='fits', hdu='FRAMES')

## Find jura exposures not in daily, daily exposures not in jura

In [ ]:
assert (np.unique(jura_exposures['EXPID']) == sorted(jura_exposures['EXPID'])).all()
assert (np.unique(daily_exposures['EXPID']) == sorted(daily_exposures['EXPID'])).all()
assert (np.unique(jura_frames['EXPID']) == sorted(jura_exposures['EXPID'])).all()
assert (np.unique(daily_frames['EXPID']) == sorted(daily_exposures['EXPID'])).all()

In [ ]:
first_jura_exposure, last_jura_exposure = jura_exposures['EXPID'].min(), jura_exposures['EXPID'].max()
first_jura_exposure, last_jura_exposure

In [ ]:
first_jura_night = jura_exposures['NIGHT'][jura_exposures['EXPID'] == first_jura_exposure].min()
last_jura_night = jura_exposures['NIGHT'][jura_exposures['EXPID'] == last_jura_exposure].max()
first_jura_night, last_jura_night

In [ ]:
jura_tiles['LASTNIGHT'].min(), jura_tiles['LASTNIGHT'].max()

In [ ]:
jura_expid_set = frozenset(jura_exposures['EXPID'].tolist())
daily_expid_set = frozenset(daily_exposures['EXPID'].tolist())

In [ ]:
jura_not_in_daily = jura_expid_set - daily_expid_set
jura_not_in_daily

In [ ]:
daily_not_in_jura = daily_expid_set - jura_expid_set
# daily_not_in_jura

## Patch frames

We don't necessarily want to change the values in the `FRAMES` table, just make sure it is consistent with the `EXPOSURES` table.

In [ ]:
daily_frames_join = Table()
daily_frames_join['FRAMEID'] = np.array([100*row['EXPID'] + cameraid(row['CAMERA']) for row in daily_frames])
daily_frames_join['DAILY_INDEX'] = np.arange(len(daily_frames))

In [ ]:
jura_frames_join = Table()
jura_frames_join['FRAMEID'] = np.array([100*row['EXPID'] + cameraid(row['CAMERA']) for row in jura_frames])
jura_frames_join['JURA_INDEX'] = np.arange(len(jura_frames))

In [ ]:
j = join(daily_frames_join, jura_frames_join, join_type='outer', keys='FRAMEID')
# j

In [ ]:
daily_frames_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['DAILY_INDEX']
jura_frames_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['JURA_INDEX']

In [ ]:
daily_frames_patched = daily_frames.copy()
for column in daily_frames_patched.colnames:
    if hasattr(daily_frames_patched[column], 'mask') and not column.startswith('TSNR2_'):
        print(f"Patching {column}...")
        daily_frames_patched[column][daily_frames_index] = jura_frames[column][jura_frames_index]
        daily_frames_patched[column].mask[daily_frames_index] = False
# daily_frames_patched

## Patch exposures

We want to only *patch* and rows that:

* Appear in `jura`.
* Have `NIGHT >= first_jura_night`.
* Have `EFFTIME_SPEC > 0`.

We want to only *load* rows that:

* Have `NIGHT >= first_jura_night`.
* Have `EFFTIME_SPEC > 0`.

which is slightly different. However we don't want to *remove* rows that *don't* satisfy these criteria.

In [ ]:
daily_exposures_join = Table()
daily_exposures_join['EXPID'] = daily_exposures['EXPID']
daily_exposures_join['NIGHT'] = daily_exposures['NIGHT']
daily_exposures_join['EFFTIME_SPEC'] = daily_exposures['EFFTIME_SPEC']
daily_exposures_join['DAILY_INDEX'] = np.arange(len(daily_exposures))

In [ ]:
jura_exposures_join = Table()
jura_exposures_join['EXPID'] = jura_exposures['EXPID']
jura_exposures_join['JURA_INDEX'] = np.arange(len(jura_exposures))

In [ ]:
j = join(daily_exposures_join, jura_exposures_join, join_type='outer', keys='EXPID')
j

In [ ]:
daily_exposures_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask) & (j['NIGHT'] >= first_jura_night) & (j['EFFTIME_SPEC'] > 0)]['DAILY_INDEX']
jura_exposures_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)& (j['NIGHT'] >= first_jura_night) & (j['EFFTIME_SPEC'] > 0)]['JURA_INDEX']

In [ ]:
daily_exposures_patched = daily_exposures.copy()
can_patch = ('NIGHT', 'EXPID', 'TILEID', 'TILERA', 'TILEDEC', 'MJD', 'SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'EXPTIME', 'GOALTIME', 'GOALTYPE', 'MINTFRAC', 'AIRMASS', 'EBV', 'SEEING_ETC', 'EFFTIME_ETC',
             'TRANSPARENCY_GFA', 'SEEING_GFA', 'FIBER_FRACFLUX_GFA', 'FIBER_FRACFLUX_ELG_GFA', 'FIBER_FRACFLUX_BGS_GFA', 'FIBERFAC_GFA', 'FIBERFAC_ELG_GFA', 'FIBERFAC_BGS_GFA', 'AIRMASS_GFA', 'SKY_MAG_AB_GFA',
             'EFFTIME_GFA', 'EFFTIME_DARK_GFA', 'EFFTIME_BRIGHT_GFA', 'EFFTIME_BACKUP_GFA')
for column in ['TILERA', 'TILEDEC', 'MJD', 'SURVEY'] + [c for c in daily_exposures_patched.colnames if hasattr(daily_exposures_patched[c], 'mask') and c in can_patch]:
    print(f"Patching {column}...")
    daily_exposures_patched[column][daily_exposures_index] = jura_exposures[column][jura_exposures_index]
    if hasattr(daily_exposures_patched[column], 'mask'):
        daily_exposures_patched[column].mask[daily_exposures_index] = False
daily_exposures_patched

### After patching are there still missing data?

In [ ]:
assert not (daily_exposures_patched['TILERA'] == daily_exposures['TILERA']).all()
assert not (daily_exposures_patched['TILEDEC'] == daily_exposures['TILEDEC']).all()
assert not (daily_exposures_patched['MJD'] == daily_exposures['MJD']).all()
assert not (daily_exposures_patched['SURVEY'] == daily_exposures['SURVEY']).all()
assert (daily_exposures_patched['PROGRAM'] == daily_exposures['PROGRAM']).all()
assert (daily_exposures_patched['FAPRGRM'] == daily_exposures['FAPRGRM']).all()
assert (daily_exposures_patched['FAFLAVOR'] == daily_exposures['FAFLAVOR']).all()

In [ ]:
missing_mjd = np.where((daily_exposures_patched['NIGHT'] >= first_jura_night) & (daily_exposures_patched['EFFTIME_SPEC'] > 0) & (daily_exposures_patched['MJD'] < 50000))[0]
for row in daily_exposures_patched[missing_mjd]:
    raw_data_file = os.path.join(os.environ['DESI_SPECTRO_DATA'], "{0:08d}".format(row['NIGHT']), "{0:08d}".format(row['EXPID']), "desi-{0:08d}.fits.fz".format(row['EXPID']))
    with fits.open(raw_data_file, mode='readonly') as hdulist:
        mjd_obs = hdulist['SPEC'].header['MJD-OBS']
    print("INFO: tile {0:d} exposure {1:d} has MJD-OBS = {2:f} in {3}!".format(row['TILEID'], row['EXPID'], mjd_obs, raw_data_file))
    w = np.where(daily_exposures_patched['EXPID'] == row['EXPID'])[0]
    assert len(w) == 1
    daily_exposures_patched['MJD'][w] = mjd_obs
    print(daily_exposures_patched[w][['NIGHT', 'EXPID', 'TILEID', 'MJD']])

In [ ]:
still_missing_mjd = np.where((daily_exposures_patched['NIGHT'] >= first_jura_night) & (daily_exposures_patched['EFFTIME_SPEC'] > 0) & (daily_exposures_patched['MJD'] < 50000))[0]
daily_exposures_patched[still_missing_mjd]

## Patch tiles

Similar to the discussion above, we want `LASTNIGHT >= first_jura_night`.

In [ ]:
assert (np.unique(jura_exposures['TILEID']) == sorted(jura_tiles['TILEID'])).all()
assert (np.unique(daily_exposures['TILEID']) == sorted(daily_tiles['TILEID'])).all()

In [ ]:
daily_tiles_join = Table()
daily_tiles_join['TILEID'] = daily_tiles['TILEID']
daily_tiles_join['LASTNIGHT'] = daily_tiles['LASTNIGHT']
daily_tiles_join['EFFTIME_SPEC'] = daily_tiles['EFFTIME_SPEC']
daily_tiles_join['DAILY_INDEX'] = np.arange(len(daily_tiles))

In [ ]:
jura_tiles_join = Table()
jura_tiles_join['TILEID'] = jura_tiles['TILEID']
jura_tiles_join['JURA_INDEX'] = np.arange(len(jura_tiles))

In [ ]:
j = join(daily_tiles_join, jura_tiles_join, join_type='outer', keys='TILEID')
j

In [ ]:
# Apparently every tile in daily also appears in jura, so j['DAILY_INDEX'] doesn't need to be masked.
daily_tiles_index = j[(~j['JURA_INDEX'].mask) & (j['LASTNIGHT'] >= first_jura_night) & (j['EFFTIME_SPEC'] > 0)]['DAILY_INDEX']
jura_tiles_index = j[(~j['JURA_INDEX'].mask) & (j['LASTNIGHT'] >= first_jura_night) & (j['EFFTIME_SPEC'] > 0)]['JURA_INDEX']

In [ ]:
daily_tiles_patched = daily_tiles.copy()
# for column in jura_tiles.colnames:
# for column in ('TILEID', 'SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'TILERA', 'TILEDEC', 'EFFTIME_ETC', 'EFFTIME_GFA', 'GOALTIME', 'GOALTYPE', 'MINTFRAC'):
for column in ('PROGRAM', ):
    daily_tiles_patched[column][daily_tiles_index] = jura_tiles[column][jura_tiles_index]
    daily_tiles_patched[column].mask[daily_tiles_index] = False
daily_tiles_patched

### After patching are there still missing data?

Also does `GOALTYPE` actually need to be patched? Should we patch `EFFTIME_SPEC`?

In [ ]:
missing_program = np.where((daily_tiles_patched['LASTNIGHT'] >= first_jura_night) & (daily_tiles_patched['EFFTIME_SPEC'] > 0) & (daily_tiles_patched['PROGRAM'].mask))[0]
daily_tiles_patched[missing_program]

## Write out the patched files

In [ ]:
daily_tiles_patched.write(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'tiles-daily-patched-with-jura.csv'), format='ascii.csv', overwrite=True)

In [ ]:
daily_exposures_patched.write(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'exposures-daily-patched-with-jura.csv'), format='ascii.csv', overwrite=True)

In [ ]:
daily_exposures_fits = fits.HDUList([fits.PrimaryHDU(), fits.table_to_hdu(daily_exposures_patched), fits.table_to_hdu(daily_frames_patched)])

In [ ]:
daily_exposures_fits.info()

In [ ]:
daily_exposures_fits.writeto(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'exposures-daily-patched-with-jura.fits'), overwrite=True)